In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from datetime import datetime

In [2]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

In [3]:
spark = SparkSession.builder\
    .appName('autism-files')\
    .config(conf=conf)\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-86fe6291-aa42-40b7-8a66-215b63f5173b;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 343ms :: artifacts dl 19ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

22/10/12 00:08:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.parquet(f"s3a://projeto-pi-datalake-trusted/autism-region")

22/10/12 00:08:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df.printSchema()

root
 |-- id_paciente: string (nullable = true)
 |-- idade_atual: string (nullable = true)
 |-- idade_descoberta: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- grau: string (nullable = true)
 |-- sensibilidade_sentidos: string (nullable = true)
 |-- agressivo: string (nullable = true)
 |-- hiperativo: string (nullable = true)
 |-- movimentos_repetitivos: string (nullable = true)
 |-- baixa_concentracao: string (nullable = true)
 |-- hiperfoco: string (nullable = true)
 |-- necessidade_rotina: string (nullable = true)
 |-- dificuldade_imaginacao: string (nullable = true)
 |-- introvertido: string (nullable = true)
 |-- tipo_autismo: string (nullable = true)
 |-- tipo_autismo_descricao: string (nullable = true)
 |-- classe: string (nullable = true)
 |-- classe-descricao: string (nullable = true)
 |-- empregado: boolean (nullable = true)
 |-- per_capita: double (nullable = true)
 |-- quantidade_familia: long (nullable = true)
 |-- total_salario: long (nullable = true

In [6]:
df.head()

22/10/12 00:08:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(id_paciente='8647DC55C75FE6B49B05AB4F6E2C4EF604EFE2672FC477BB76C0114F8F', idade_atual='44', idade_descoberta='29', genero='B850D4F5937B17CE7D2632EE42F6F88AFE6BCAE572821F2612C20B0CD3', grau='severo', sensibilidade_sentidos='True', agressivo='True', hiperativo='False', movimentos_repetitivos='False', baixa_concentracao='True', hiperfoco='True', necessidade_rotina='True', dificuldade_imaginacao='True', introvertido='False', tipo_autismo='Transtorno do espectro Autista (TEA)', tipo_autismo_descricao='Está extremidade mais branda do espectro autista, pois a inteligência pode ser alta e a capacidade de realizar as atividades diárias é preservada. No entanto, a dificuldade na interação social é muito comum.', classe='B', classe-descricao='Baixa classe alta', empregado=True, per_capita=9977.0, quantidade_familia=1, total_salario=9977, PIB_nacional=8700000000000, PIB_percent_BR=5.3, PIB_regional=1195000000000, area_regiao=3853676948, densidade_demografica_regiao=4.12, gentilico_regiao='nort

In [7]:
execution_date = datetime(2022, 5, 29)

In [8]:
df_grouped = (
    df.select(
            'regiao', 'tipo_autismo', 'classe', 'tipo_autismo_descricao',
            'raca_predominante_regiao', 'genero', 'grau', 'classe-descricao', 
            'pais', 'gentilico_regiao', f.col('agressivo').cast('boolean'), f.col('hiperfoco').cast('boolean'), 
            f.col('hiperativo').cast('boolean'), f.col('sensibilidade_sentidos').cast('boolean'),
            f.col('movimentos_repetitivos').cast('boolean'), f.col('baixa_concentracao').cast('boolean'),
            f.col('necessidade_rotina').cast('boolean'), f.col('dificuldade_imaginacao').cast('boolean'),
            'empregado', 'per_capita', 'densidade_demografica_regiao', 'populacao_regiao', 'quantidade_familia',
            'total_salario'
        )
        .groupby(
            'regiao', 'tipo_autismo', 'classe', 'tipo_autismo_descricao',
            'raca_predominante_regiao', 'genero', 'grau', 'classe-descricao', 'pais',
            'gentilico_regiao'
        )
        .agg(
            f.count(f.when(f.col('agressivo'), f.col('agressivo'))).alias('agressivo'),
            f.count(f.when(f.col('hiperfoco'), f.col('hiperfoco'))).alias('hiperfoco'),
            f.count(f.when(f.col('hiperativo'), f.col('hiperativo'))).alias('hiperativo'),
            f.count(f.when(f.col('sensibilidade_sentidos'), f.col('sensibilidade_sentidos'))).alias('sensibilidade_sentidos'),
            f.count(f.when(f.col('movimentos_repetitivos'), f.col('movimentos_repetitivos'))).alias('movimentos_repetitivos'),
            f.count(f.when(f.col('baixa_concentracao'), f.col('baixa_concentracao'))).alias('baixa_concentracao'),
            f.count(f.when(f.col('necessidade_rotina'), f.col('necessidade_rotina'))).alias('necessidade_rotina'),
            f.count(f.when(f.col('dificuldade_imaginacao'), f.col('dificuldade_imaginacao'))).alias('dificuldade_imaginacao'),
            f.count(f.when(f.col('empregado'), f.col('empregado'))).alias('empregado'),
            f.mean('per_capita').alias('per_capita'),
            f.mean('densidade_demografica_regiao').alias('densidade_demografica_regiao'),
            f.mean('populacao_regiao'),
            f.mean('quantidade_familia').alias('quantidade_familia_media'),
            f.max('quantidade_familia').alias('quantidade_familia_maxima'),
            f.min('quantidade_familia').alias('quantidade_familia_minima'),
            f.mean('total_salario').alias('total_salario_medio'),
            f.max('total_salario').alias('total_salario_maximo'),
            f.min('total_salario').alias('total_salario_minimo')
        )
        .withColumn('year', f.lit(execution_date.year))
        .withColumn('month', f.lit(execution_date.month))
        .withColumn('day', f.lit(execution_date.day))
)

In [9]:
df_grouped.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- tipo_autismo: string (nullable = true)
 |-- classe: string (nullable = true)
 |-- tipo_autismo_descricao: string (nullable = true)
 |-- raca_predominante_regiao: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- grau: string (nullable = true)
 |-- classe-descricao: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- gentilico_regiao: string (nullable = true)
 |-- agressivo: long (nullable = false)
 |-- hiperfoco: long (nullable = false)
 |-- hiperativo: long (nullable = false)
 |-- sensibilidade_sentidos: long (nullable = false)
 |-- movimentos_repetitivos: long (nullable = false)
 |-- baixa_concentracao: long (nullable = false)
 |-- necessidade_rotina: long (nullable = false)
 |-- dificuldade_imaginacao: long (nullable = false)
 |-- empregado: long (nullable = false)
 |-- per_capita: double (nullable = true)
 |-- densidade_demografica_regiao: double (nullable = true)
 |-- avg(populacao_regiao): double (nul

In [10]:
df_grouped.show(vertical=True, truncate=False)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 regiao                       | F2EE3F4F563C93BD81FB979DA4F4B4B61D0247FE09A1F4E8A8ED3422892B022B69044F12177E3C97B1EA9B6884418785ABC9F9511A99FF7CFC9C3AFC5315B959EF79FB1CE5AD7EDBA2B03F38C81DFD7D56E2AECC7937                                                                                                                               
 tipo_autismo                 | Transtorno do espectro Autista (TEA)                                                                                                                                                                                                                                                                       
 cla

In [11]:
df_grouped.write.mode('overwrite').partitionBy('year', 'month', 'day')\
    .csv('s3a://projeto-pi-datalake-analytics/autism-region', header=True)

22/10/12 00:08:31 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


22/10/12 00:08:37 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/10/12 00:08:37 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
